In [ ]:
from path_dataloader import WSIData
from PSO import PSO
from formula import PSO_train
import time

import torch
import torchvision.transforms as transforms
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
begin_time=time.time()
dataset_path='/root/autodl-tmp/data/lung/lung_colon_image_set/lung_image_sets'
p_train=0.8 # the part that training data is
batch_size=32
Num_worker=11
particle_num=4
generation_num=10
training_epoch=5

# Dataset preparation
transform_train = transforms.Compose([
    transforms.RandomCrop((200,200)),
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
])

dataset=WSIData(file_path=dataset_path,data_name="lc25000",transform=transform_train,test_flag=0,img_color_space="lab")
dataset.get_detailed_info()
dataset.get_class_count()

train_size, test_size = int(p_train * len(dataset)), int((len(dataset)-int(p_train * len(dataset))))
print(train_size, test_size)

trainset, testset= torch.utils.data.random_split(dataset, [train_size, test_size])
trainloader = torch.utils.data.DataLoader(trainset, shuffle =True, batch_size=batch_size,num_workers=Num_worker,drop_last=True) 
testloader = torch.utils.data.DataLoader(testset, shuffle = False,num_workers=Num_worker,batch_size=128,drop_last=True)
print("Datasets have been loaded")

# PSO generation
group=PSO(part_num=particle_num,dim=7,minn=0,maxn=128,maxv=10,c1=2,c2=2)
group.info_print()


# Info output at the end of processing








In [ ]:
for i in range(generation_num):
    PSO_train(group,trainloader,testloader,start_training=1, epoch=training_epoch)
    group.update()
    group.info_print()
print("Total time: ", time.time()-begin_time)    
for i in group.group:
    print(i.histo)
    